### Importing libraries and modules

In [22]:
import networkx as nx
import os
import pandas as pd
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
from collections import Counter
from networkx.linalg.graphmatrix import adjacency_matrix
import numpy as np
from statistics import mean
from scipy.stats import pearsonr

# Loading data pathes and I/O functions from script
from scripts.io import load_movie_titles, load_raw_bipartite, save_projection, load_projection, save_edgelist, projection_path

import scripts.recommend as rec

### Loading bipartite graph and movie titles

In [23]:
title_dict, node_dict = load_movie_titles("movie-titles.txt")
G = load_raw_bipartite("full_bipartite.p")

# Split the graph into 2 sets: user and movie nodes
user_nodes, movie_nodes = nx.algorithms.bipartite.basic.sets(G)

Movie titles loaded.
Graph loaded.


### Simple weights projections

In [24]:
# Projecting on users
simple_weights_users_path = "simple_weights_users.p"

if os.path.exists(projection_path+simple_weights_users_path):
    simple_weights_users = load_projection(simple_weights_users_path)
else:
    simple_weights_users = bipartite.weighted_projected_graph(G, user_nodes, ratio=True)
    save_projection(simple_weights_users, simple_weights_users_path)

# Projecting on movies
simple_weights_movies_path = "simple_weights_movies.p"

if os.path.exists(projection_path+simple_weights_movies_path):
    simple_weights_movies = load_projection(simple_weights_movies_path)
else:
    simple_weights_movies = bipartite.weighted_projected_graph(G, movie_nodes, ratio=True)
    save_projection(simple_weights_movies, simple_weights_movies_path)

Projection loaded.
Projection loaded.


### Research Question
How can we recommend movies, given movies you like?

#### Idea for an algorithm where projections take ratings into account in some way:
1. Get input of movies you like, M, which have ratings by users U
2. Project U and M onto U to get U1
3. Project U1, and all movies they rated, onto the movies to get M1
4. Recommend, from M1, the highest weight neighbor(s) of M.

#### Other ideas:
* Sample users who liked the movie(s), project onto their rated movies, recommend highest weight neighbor of liked movies.  
* Construct similar movies to the ones you like, use this to find users like you, and iterate to converge on movies you will like.  

Potential problem: if our sample of users is too large, we are likely to just recommend the most rated movies, not specific movies you would like. To test this, we could plot correlation between movie degree and likelihood to recommend.  

Does backboning M1 improve recommendations?  

How can we take movie genre into account for recommendation?  

Are our movie recommendations associated with genre? i.e. does M1 have high genre
homophily?  

Are our recommendations largely popular or niche movies and why?  

For evaluation, can we use cross validation by comparing users’ ratings to how likely we are to recommend each rated movie, based on a sample of movies they like?  

# Our Projection Methods

## Rating Allocation
Projection algorithm inspired by resource allocation, where directed edge weight from movie1 to movie2 is computed by summing over all users who rated both movies, multiplying their rating of movie1, normalized by all movie1's ratings, with their rating of movie2, normalized by the users total ratings.

$$RA_{m1,m2} = \sum_{u \in N_{m1} \cap N_{m2}} \frac{w_{m1,u}}{\sum{w_{m1}}}\frac{w_{u,m2}}{\sum{w_{u}}}$$ 

this is computed for all $m1,m2$ pairs of movies, excluding self-loops, to produce a directed movie graph.

An issue with this approach: Ratings below average (e.g. 1) increase weight compared to no rating, which seems intuitively wrong.  
We solve this by replacing no rating with the average rating, for both $w_{m1,u}$ and $w_{u,m2}$.



In [25]:
def find_common_neighbors(u, v):
    N_u = G.neighbors(u)
    N_v = G.neighbors(v)
    return set(N_u) & set(N_v)

def find_neighbor_difference(u, v):
    N_u = G.neighbors(u)
    N_v = G.neighbors(v)
    return set(N_u).difference(set(N_v))

def find_non_neighboring_nodes(u, v):
    N_u = G.neighbors(u)
    N_v = G.neighbors(v)
    return set(user_nodes).difference(set(N_u).union(set(N_v)))

# Old approach (incorrectly using degree)
def rating_allocation_edge_weight(G, u, v, degree_u):
    N_u_v = find_common_neighbors(u,v)
    weight_u_v = 0
    for n in N_u_v:
        w_u_n = G.get_edge_data(u,n)['weight']
        w_n_v = G.get_edge_data(n,v)['weight']
        weight_u_v += w_u_n / degree_u * w_n_v / G.degree(n)
    return weight_u_v

# The algorithm works in 3 stages:
#   1. Iterate over the common neighbors of the movies, calculate weights.
#   2. Iterate over the differences between the neighbor sets of the movies. Calculate the weights.
#   3. Now, there is no need to iterate over the users that are not in either movie's neighbor set. As
#      it will be explained later, it is a one time operation.
def rating_allocation_edge_weight_adjusted(G, u, v):
    N_u_v = find_common_neighbors(u,v)
    Dif_u_v = find_neighbor_difference(u, v)
    Dif_v_u = find_neighbor_difference(v, u)
    Non_n_u_v = find_non_neighboring_nodes(u, v)

    # The total weight of a node is calculated as the average weight of the node times the number of edges. Since
    # the bipartite network is a clique, the number of edges are the number of user nodes.
    u_total_weight = len(user_nodes) * G.nodes[u]["average_weight"]

    number_of_movies = len(movie_nodes)
    number_of_users = len(user_nodes)
    number_of_movies_inverse = 1 / number_of_movies  # Will be important later.
    number_of_users_inverse = 1 / number_of_users

    weight_u_v = 0
    # Case 1: The intersection of the neighbors. Need the find the weights of both the edges since they are
    # different from the default average weight.
    for n in N_u_v:
        w_u_n = G.get_edge_data(u,n)['weight']
        w_n_v = G.get_edge_data(n,v)['weight']
        weight_u_v += (w_u_n / u_total_weight * w_n_v / (number_of_movies * G.nodes[n]["average_weight"]))

    # Case 2a: The difference of the neighboring sets of u and v. Notice the second term in the calculation. This
    # is a simplification of (G.nodes[n]["average_weight"] / (G.nodes[n]["average_weight"] * number_of_movies)).
    # There is no real edge between n and v, so we assume the edge weight is the average weight for node v.
    for n in Dif_u_v:
        w_u_n = G.get_edge_data(u, n)['weight']
        weight_u_v += (w_u_n / u_total_weight) * number_of_movies_inverse

    # Case 2b: Similar to case 2a.
    for n in Dif_v_u:
        w_v_n = G.get_edge_data(n, v)['weight']
        weight_u_v += number_of_users_inverse * (w_v_n / (number_of_movies * G.nodes[n]["average_weight"]))

    # Now, we can do the simplification shown before for both terms, since we know the edge weights will be
    # equal to their average weights because there is no real node between u -> n and n -> v. We do this operation
    # for the number of users that are not shared in both movies' neighboring sets.
    weight_u_v += len(Non_n_u_v) * (number_of_users_inverse * number_of_movies_inverse)
    return weight_u_v

def rating_allocation_projection(G, movie_nodes):
    rating_allocation_graph = nx.DiGraph()
    for i, u in enumerate(movie_nodes):
        for v in movie_nodes:

            # Prevent self-loops
            if v == u:
                continue

            #Change this for testing
            edge_weight = rating_allocation_edge_weight_adjusted(G, u, v)
            rating_allocation_graph.add_edge(u, v, weight=edge_weight)
        
        # print progress
        print(f"{i/len(movie_nodes):.0%}")
        print(rating_allocation_graph[u][v])
    return rating_allocation_graph

In [26]:
# Possible solution to problem:
#   1. Compute the average weight for each node and put in a dictionary. Done
#   2. Iterate over the user nodes and check if they have a connection with the movies.
#   3. If they do not, add an edge with average weight.
#   4. To normalize, we add average rating * number of pretended edges at the end

print("User amount: ", len(user_nodes))
print("Movie amount: ", len(movie_nodes))

def adjust_nodes(nodes):
    for m in nodes:
        number_of_neighbors = sum(1 for _ in G.neighbors(m))

        total_weight = 0
        for v in G.neighbors(m):
            total_weight += G.get_edge_data(m, v)["weight"]
        average_weight = total_weight / number_of_neighbors

        G.nodes[m]["total_weight"] = total_weight
        G.nodes[m]["average_weight"] = average_weight
        G.nodes[m]["number_of_neighbors"] = number_of_neighbors

adjust_nodes(user_nodes)
adjust_nodes(movie_nodes)

User amount:  943
Movie amount:  1682


## Rating Allocation with genres

In [27]:
# Insert the genre information into the movie nodes.
genres = dict()

# Load genre information in a dict
def create_genre_dict(genre_dict):
    with open("./data/raw/genres.txt", 'r') as genre_info:
        for line in genre_info:
            fields = line.split('|')
            genre_dict[int(fields[1])] = fields[0]

create_genre_dict(genres)

# Load genre information of movies into the movie nodes
def load_genres(graph, genre_dict):
    with open("./data/raw/movie-titles.txt", 'r') as movie_genres:
        for line in movie_genres:
            fields = line.strip().split('|')
            movie_node = int(fields[0])
            if not movie_node in graph:
                continue
            for index in range(0,19):
                graph.nodes[movie_node][genre_dict[index]] = int(fields[index+5])

            # Also adding genres as numpy array
            graph.nodes[movie_node]['genres'] = np.array(fields[5:24], dtype=int)

load_genres(G, genres)

In [28]:
# Testing genre correlations
nodes = 1, 201
r, p = pearsonr(G.nodes[nodes[0]]['genres'], G.nodes[nodes[1]]['genres'])
r, p, G.nodes[nodes[0]]['genres'], G.nodes[nodes[1]]['genres']

(0.1304372986874877,
 0.5945550202553832,
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]))

In [29]:
# Adding total and average weight as node attribute
def add_weight_attributes(G, nodes):

    for node in nodes:
        neighbors = set(G.neighbors(node))
        number_of_neighbors = len(neighbors)
        total_weight = 0

        for v in G.neighbors(node):
            total_weight += G.get_edge_data(node, v)["weight"]
        average_weight = total_weight / number_of_neighbors

        # For each non-neighbor, add average weight to total weight
        number_of_non_neighbors = len(G.nodes) - len(nodes) - number_of_neighbors
        total_weight += number_of_non_neighbors * average_weight

        G.nodes[node]["total_weight"] = total_weight
        G.nodes[node]["average_weight"] = average_weight

# Needs to be called on users and movies seperately
add_weight_attributes(G, user_nodes)
add_weight_attributes(G, movie_nodes)

In [30]:
def chris_rating_allocation_projection(G, movie_nodes):
    """
    This works, but is slower than Efe's approach
    """
    user_nodes = {node for node in G.nodes if node not in movie_nodes}

    rating_allocation_graph = nx.DiGraph()

    for i, m1 in enumerate(movie_nodes):

        m1_neighbors = set(G.neighbors(m1))

        for m2 in movie_nodes:
            # Prevent self-loops
            if m1 == m2:
                continue

            weight_m1_m2 = 0

            for user in user_nodes:
            
                if user in m1_neighbors:
                    w_m1_user = G.get_edge_data(m1,user)['weight']
                else:
                    w_m1_user = G.nodes[m1]['average_weight']

                if user in G[m2]:
                    w_user_m2 = G.get_edge_data(user,m2)['weight']
                else:
                    w_user_m2 = G.nodes[user]['average_weight']

                weight_m1_m2 += (w_m1_user / G.nodes[m1]['total_weight']) * (w_user_m2 / G.nodes[user]['total_weight'])
            rating_allocation_graph.add_edge(m1, m2, weight=weight_m1_m2)

        # print progress
        print(f"{i/len(movie_nodes):.0%}")
        print(m1,m2,rating_allocation_graph[m1][m2]['weight'])

    return rating_allocation_graph

#chris_rating_allocation_projection(G, movie_nodes)

In [31]:
def find_common_neighbors(u, v):
    N_u = G.neighbors(u)
    N_v = G.neighbors(v)
    return set(N_u) & set(N_v)

def find_neighbor_difference(u, v):
    N_u = G.neighbors(u)
    N_v = G.neighbors(v)
    return set(N_u).difference(set(N_v))

def find_non_neighboring_nodes(u, v):
    N_u = G.neighbors(u)
    N_v = G.neighbors(v)
    return set(user_nodes).difference(set(N_u).union(set(N_v)))

# Old approach (incorrectly using degree)
def rating_allocation_edge_weight(G, u, v, degree_u):
    N_u_v = find_common_neighbors(u,v)
    weight_u_v = 0
    for n in N_u_v:
        w_u_n = G.get_edge_data(u,n)['weight']
        w_n_v = G.get_edge_data(n,v)['weight']
        weight_u_v += w_u_n / degree_u * w_n_v / G.degree(n)
    return weight_u_v

def rating_allocation_edge_weight_adjusted(G, m1, m2):
    m1_total_weight = G.nodes[m1]["total_weight"]
    weight_m1_m2 = 0
    # Case 1: The intersection of the neighbors. 
    # Need the find the weights of both the edges since they are different from the default average weight.
    for user in find_common_neighbors(m1,m2):
        w_m1_user = G.get_edge_data(m1,user)['weight']
        w_user_m2 = G.get_edge_data(user,m2)['weight']
        weight_m1_m2 += w_m1_user / m1_total_weight * w_user_m2 / G.nodes[user]["total_weight"]

    # Case 2a: The difference of the neighboring sets of u and v. 
    # There is no real edge between user and m2, so we assume the edge weight is the average rating devided by the total ratings of user.
    for user in find_neighbor_difference(m1, m2):
        w_m1_user = G.get_edge_data(m1, user)['weight']
        weight_m1_m2 += (w_m1_user / m1_total_weight) * (G.nodes[user]["average_weight"] / G.nodes[user]["total_weight"])

    # Case 2b: Similar to case 2a, but instead use average for weight from m1 to user.
    for user in find_neighbor_difference(m2, m1):
        w_user_m2 = G.get_edge_data(user, m2)['weight']
        weight_m1_m2 += (G.nodes[m1]["average_weight"] / m1_total_weight) * (w_user_m2 / G.nodes[user]["total_weight"])
    
    # Using average weights between m1 and user, and user and m2.
    for user in find_non_neighboring_nodes(m1, m2):
        weight_m1_m2 += (G.nodes[m1]["average_weight"] / m1_total_weight) * (G.nodes[user]["average_weight"] / G.nodes[user]["total_weight"])

    return weight_m1_m2

def rating_allocation_projection(G, movie_nodes):
    """
    New approach.
    Uses average rating of m1 for m1->user if user didn't rate m1, 
    and avg. rating of user for user->m2 if user didn't rate m2.
    """
    rating_allocation_graph = nx.DiGraph()
    for i, m1 in enumerate(movie_nodes):
        for m2 in movie_nodes:
            # Prevent self-loops
            if m2 == m1:
                continue
            edge_weight = rating_allocation_edge_weight_adjusted(G, m1, m2)
            rating_allocation_graph.add_edge(m1, m2, weight=edge_weight)
        # print progress
        print(f"{i/len(movie_nodes):.0%}")
        # print(m1,m2,rating_allocation_graph[m1][m2]['weight'])
    return rating_allocation_graph

def rating_genre_allocation_projection(G, movie_nodes, genre_weight=0):
    """
    Adds optional genre_weight parameter [0,1], which is weight of genre correlation compared to rating_allocation
    """
    num_movies = len(movie_nodes)
    rating_allocation_graph = nx.DiGraph()
    for i, m1 in enumerate(movie_nodes):
        for m2 in movie_nodes:
            # Prevent self-loops
            if m2 == m1:
                continue
            edge_weight = rating_allocation_edge_weight_adjusted(G, m1, m2)

            if genre_weight > 0:

                # Get pearson correlation between movie genres
                r, _ = pearsonr(G.nodes[m1]['genres'], G.nodes[m2]['genres'])  

                # Ignore negative correlations
                r = max(0, r)  

                # New edge weight is sum with correlation, weighed by genre_weight
                edge_weight = genre_weight * r + (1-genre_weight) * edge_weight

            rating_allocation_graph.add_edge(m1, m2, weight=edge_weight)
        # print progress
        print(f"{i/num_movies:.0%}")
        # print(m1,m2,rating_allocation_graph[m1][m2]['weight'])
    return rating_allocation_graph

In [32]:
# Saving/loading rating allocation projection with genres
rating_genre_allocation_path = "rating_genre_allocation_movies.p"

if os.path.exists(projection_path+rating_genre_allocation_path):
    rating_genre_allocation_movies = load_projection(rating_genre_allocation_path)
else:
    # Takes a VERY long time
    rating_genre_allocation_movies = rating_genre_allocation_projection(G, movie_nodes, genre_weight=0.5)
    save_projection(rating_genre_allocation_movies, rating_genre_allocation_path)

Projection loaded.


In [33]:
# Saving/loading rating allocation projection on movies
rating_allocation_movies_path = "rating_allocation_movies.p"

if os.path.exists(projection_path+rating_allocation_movies_path):
    rating_allocation_movies = load_projection(rating_allocation_movies_path)
else:
    # Takes VERY long
    rating_allocation_movies = rating_genre_allocation_projection(G, movie_nodes)
    save_projection(rating_allocation_movies, rating_allocation_movies_path)



Projection loaded.


## Saving edge lists for visualization with Gephi

In [34]:
# TODO: Getting 50 highest degree movies

# TODO: Pass these to save_edgelist
# save_edgelist(50, rating_allocation_movies, "rating_allocation_movies_edges", title_dict, overwrite=True)
# save_edgelist(50, simple_weights_movies, "simple_weights_movies_edges", title_dict)

## Recommendation

In [35]:
# liked_movie_list = ['Die Hard (1988)', 'Star Wars (1977)']
# graph = rating_allocation_movies
# liked_movie_node_list = [node_dict[liked_movie_title] for liked_movie_title in liked_movie_list]
# neighbors_weights = dict()
# for liked_movie_node in liked_movie_node_list:  # For each of the liked movies
#     for node, neighbor, attr_dict in graph.edges(liked_movie_node, data=True):  # For each of its edges
        
#         # Avoid edges to liked movies
#         if neighbor in liked_movie_node_list:
#             continue
        
#         # Append weight to neighbor to dict of all weights
#         neighbors_weights.setdefault(neighbor, []).append(attr_dict['weight'])  

# # Average over all weights for each movie
# avg_neighbors_weights = [(node, mean(weights)) for node, weights in neighbors_weights.items()]


In [36]:
# # To recommend, find highest weight movie from movie you like

# def k_highest_weight_neighbors(k, liked_movie_title, graph):
    
#     liked_movie_node = node_dict[liked_movie_title]
#     edges = list(graph.edges(liked_movie_node, data=True))

#     neighbors_weights = [(neighbor, weight['weight']) for node, neighbor, weight in edges]
#     neighbors_weights = sorted(neighbors_weights, reverse=True, key=lambda x: x[1])

#     neighbors_weights_dict = dict((node, find_neighbor_weights(edges[node])) for node in edges.keys())

#     n_neighbors = [(title_dict[neighbor],weight) for neighbor, weight in neighbors_weights][:k]

#     print(f"{k} highest weight neighbors of '{liked_movie_title}':")
#     return n_neighbors

# k_highest_weight_neighbors(10, 'Die Hard (1988)', rating_allocation_movies)

In [37]:
#rating_allocation_movies.edges(data=True)

In [59]:
liked_movie_list = ["Batman Forever (1995)"]
# print(edges)

In [60]:
def get_edges(liked_movie_list,d_graph):
    liked_movie_node_list = [node_dict[liked_movie_title] for liked_movie_title in liked_movie_list]
    edges = dict((liked_movie_node, list(d_graph.edges(liked_movie_node, data=True))) for liked_movie_node in liked_movie_node_list)
    return edges

edges = get_edges(liked_movie_list, rating_allocation_movies)

def get_average_weight_per_movie(edges):
    node_weights = dict()
    for node, edges_list in edges.items():
        for edge in edges_list:
            if edge[1] in edges.keys():
                continue
            if edge[1] in node_weights:
                node_weights[edge[1]].append(edge[2]['weight'])
            else:
                node_weights[edge[1]] = [edge[2]['weight']]
        average_weights = [(node,mean(weights)) for node, weights in node_weights.items()]
    return average_weights

def sort_average_weight(average_weight_edges):
    return sorted(average_weight_edges, reverse=True, key=lambda x: x[1])

def k_recommend_from_list(k, rating_allocation_movies, liked_movie_list):
    edges = get_edges(liked_movie_list, rating_allocation_movies)
    sorted_average_weights = sort_average_weight(get_average_weight_per_movie(edges))
    n_neighbors = [(title_dict[neighbor],weight) for neighbor, weight in sorted_average_weights][:k]
    return n_neighbors
print(k_recommend_from_list(20,rating_genre_allocation_movies, liked_movie_list))

[('Batman Returns (1992)', 0.5798100497880322), ('Three Musketeers, The (1993)', 0.4997857914588887), ('Cliffhanger (1993)', 0.4996075564768947), ('Rumble in the Bronx (1995)', 0.49955533854694517), ('Batman & Robin (1997)', 0.4990640798011138), ('Princess Bride, The (1987)', 0.4235724323071162), ('Men in Black (1997)', 0.4230792507523231), ('True Lies (1994)', 0.42239145169878556), ('Best Men (1997)', 0.42226412122357415), ('Evil Dead II (1987)', 0.4222279164713217), ('Batman (1989)', 0.4221651001980307), ('Hard Target (1993)', 0.42210271426732116), ('Operation Dumbo Drop (1995)', 0.42203104919718926), ('Raiders of the Lost Ark (1981)', 0.4150872277979819), ('Indiana Jones and the Last Crusade (1989)', 0.4136584370219802), ('Sting, The (1973)', 0.4136115396579264), ('Some Like It Hot (1959)', 0.41296495498982744), ('Grosse Pointe Blank (1997)', 0.41279500580062917), ('Adventures of Robin Hood, The (1938)', 0.41271518121913675), ('New York Cop (1996)', 0.4127150168965514)]


In [40]:
#k = 10
#n_neighbors = [(title_dict[neighbor],weight) for neighbor, weight in sorted_average_weights][:k]
#print(n_neighbors)

In [41]:
# For multiple likes movies, find highest average weight movie

In [ ]:
load_genres(rating_allocation_movies, genres)
load_genres(simple_weights_movies, genres)
load_genres(rating_genre_allocation_movies, genres)

assortativity_coef_dict = dict()
for genre in genres.values():
    assortativity_coef_dict[genre] = [
       nx.assortativity.attribute_assortativity_coefficient(
           rating_allocation_movies, attribute=genre
       ), nx.assortativity.attribute_assortativity_coefficient(
            simple_weights_movies, attribute=genre
        ),
        nx.assortativity.attribute_assortativity_coefficient(
            rating_genre_allocation_movies, attribute=genre
        )
    ]

df = pd.DataFrame(assortativity_coef_dict)
df.insert(0, "Projections", ["Resource Allocation","Simple Weights", "Genre Allocation"])
print(df)
